In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime

In [2]:
import methods 
import formats
import json
import os 
import re

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import methods #file containing functions to interact with Google calendar like add event, create event, etc
import json
import re

google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() #get the task service
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
client = methods.set_up_MISTRAL(calendar, tasks_list)
now = methods.current_time()

No tasks found.
Based on the information you've provided, here's your schedule for today, April 17, 2024:

Events:
1. Meeting: This event is scheduled to start at 4:00 PM and end at 5:00 PM Eastern Time.

Please note that the time is adjusted to Eastern Time Zone as indicated in your calendar event.

As for your to-do list, you haven't provided any tasks yet. If you have tasks you'd like to add, feel free to let me know!


In [3]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [4]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

In [5]:
past_conversation = get_past_conversation()

context = f"Here's my calendar and tasks for the week, store it for your reference.{calendar}\n.Today's date and time is: {now}\n"
instruction = """\nAs my virtual assistant, you are an expert with managing the calendar.
You can create, move, check, list, update, delete, and handle recurring events based on the user's requests.
When the user doesn't provide enough information, you can ask for more details, up to 3 times. When there is"Task completed." in the reponse, stop aksing.
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
Check for any conflicts before scheduling an event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions. If you aren't done, end with a question mark.
Please perform the necessary actions using the Google Calendar API. Here's the previous conversation with the user, read it and understand user preferences if any, use it to make your best judgement on scheduling.\n"""

initial_prompt = context + instruction + past_conversation

In [6]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request

can you schedule a meeting tomorrow at 9am


In [7]:
messages = [
        ChatMessage(
            role="system",
            content=initial_prompt  # System message with instructions and context
        ),
        ChatMessage(
            role="user",
            content=user_request  # User's initial request
        )
    ]


In [8]:
messages

[ChatMessage(role='system', content='Here\'s my calendar and tasks for the week, store it for your reference.{\'name\': \'Meeting\', \'datetime_start\': \'2024-04-17T16:00:00-04:00\', \'datetime_end\': \'2024-04-17T17:00:00-04:00\', \'id\': \'96fps0p7c14khvvq6gsackl92s\'}\n\n.Today\'s date and time is: 2024-04-17T02:38:09.707381Z\n\nAs my virtual assistant, you are an expert with managing the calendar.\nYou can create, move, check, list, update, delete, and handle recurring events based on the user\'s requests.\nWhen the user doesn\'t provide enough information, you can ask for more details, up to 3 times. When there is"Task completed." in the reponse, stop aksing.\nWhen the user wants to create an event or move an event and if the user hasn\'t specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.\nCheck for any conflict

In [9]:
finalized_response = False

while not finalized_response:
    response = client.chat(model="mistral-large-latest", messages=messages) 
    print(response.choices[0].message.content)
    log_message("assistant", response.choices[0].message.content)
    messages.append(ChatMessage(
        role="assistant",
        content=response.choices[0].message.content
    ))
    
    if re.compile(r'\?').search(response.choices[0].message.content):
        # If there is a question, do not finalize the response.
        # Wait for user feedback, then continue the loop.
        user_feedback = input()
        messages.append(ChatMessage(
            role="user",
            content=user_feedback
        ))
        log_message("user", user_feedback)
        
    else:
        # If there is no question, finalize the response.
        finalized_response = True

Based on the information provided, there seems to be no conflict with scheduling a meeting tomorrow at 9am. However, as a helpful and respectful AI, I would like to confirm with you before proceeding with the scheduling.

Would it be okay to schedule the meeting tomorrow at 9am?
yes
Based on the information provided, I have checked your calendar and found that there is a meeting scheduled today at 4:00 PM to 5:00 PM.

Regarding the request to schedule a meeting tomorrow at 9am, there seems to be no conflict with your calendar.

Would it be okay to schedule the meeting tomorrow at 9am?
yes
Based on the information provided, I have stored your calendar and tasks for the week in my reference.

Regarding the request to schedule a meeting tomorrow at 9am, I have checked your calendar and found that there is no conflict with your schedule.

Would it be okay to schedule the meeting tomorrow at 9am?
yes
Based on the information provided, I have stored your calendar and tasks for the week in my

In [15]:
messages

[ChatMessage(role='system', content='Here\'s my calendar and tasks for the week, store it for your reference.{\'name\': \'Meeting\', \'datetime_start\': \'2024-04-17T16:00:00-04:00\', \'datetime_end\': \'2024-04-17T17:00:00-04:00\', \'id\': \'96fps0p7c14khvvq6gsackl92s\'}\n\n.Today\'s date and time is: 2024-04-17T02:38:09.707381Z\n\nAs my virtual assistant, you are an expert with managing the calendar.\nYou can create, move, check, list, update, delete, and handle recurring events based on the user\'s requests.\nWhen the user doesn\'t provide enough information, you can ask for more details, up to 3 times. When there is"Task completed." in the reponse, stop aksing.\nWhen the user wants to create an event or move an event and if the user hasn\'t specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.\nCheck for any conflict

In [49]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text.
All dates should be in 'YYYY-MM-DD' format (unless otherwise stated), and times should be in 24-hour format 'HH:MM'. Additionally, for functions requiring a datetime range (such as 'checkSchedule' and 'ListEvents'), the datetime should be in ISO 8601 format 'YYYY-MM-DDTHH:MM:SSZ'. Use createEvent for scheduling tasks.
Available function templates to include in the response string:
- createEvent(description, date:default=today, time, duration:default=1:00)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_description, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, date, time, duration:default=1:00, frequency, until)  // frequency can have the value of DAILY, WEEKLY, MONTHLY, YEARLY
- deleteRecurringEvent(<recurringEventId>) // make sure the event_id is the recurring event id
- updateRecurringEvent(<recurringEventId>, new_description, new_duration, new_location, new_frequency, new_until) // make sure the event_id is the recurring event ID
- inviteEmail(event_id, attendees)
Make sure to fill in the event IDs from the calendar provided.
Make sure to fill in the required information, such as event names, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.
Example input: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output: 'createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1:00)'
Example input: 'Update my team meeting to be two hours'
Example output 'updateEvent(<event_id>, "", 2:00, "")
Example input: 'Create a recurring meeting every Wednesday at 2pm until the end of June'
Example output: 'createRecurringEvent("Meeting", "YYYY-MM-DD", "02:00", "1:00", "WEEKLY", "YYYYMMDD"). Make sure until parameter has no dashes and follows the format YYYYMMDD.
Example input: 'Update my recurring meeting every Wednesday at 2pm to be every month until the end of August'
Example output: 'createRecurringEvent(<RecurringEventId>, "", "02:00", "1:00", "", "MONTHLY", "YYYYMMDD"). Make sure until parameter has no dashes and follows the format YYYYMMDD.
Example input: 'Invite person@gmail.com to my Team Meeting'
Example output: 'inviteEmail(<event_id>, "person@gmail.com")'
Example input: 'Move my team meeting to tomorrow at 12pm'
Example output: 'moveEvent("event_id", 'YYYY-MM-DD', '12:00')'
Example input: 'Clear my afternoon'
Example output: 'deleteEvent("event_id");deleteEvent("event_id")'
Example input: 'Schedule all my tasks'
Example output: 'createEvent("Task 1", "YYYY-MM-DD", "10:00", 1:00);createEvent("Task 2", "YYYY-MM-DD", "11:00", 1:00);createEvent("Task 3", "YYYY-MM-DD", "12:00", 1:00)\n

Now, please generate the necessary function call(s) based on the user's request."""

In [50]:
messages.append(ChatMessage(
            role="user",
            content=instruction2)
        )

In [51]:
function_call = ''
function_call = client.chat(model="mistral-large-latest", messages=messages)
function_call=function_call.choices[0].message.content
print(function_call)



createEvent("Meeting", "2024-04-18", "09:00", 1:00)


In [52]:
# Assuming the initial string of function calls is contained in a variable named function_call
function_calls = [line.strip() for line in function_call.split(';')] 
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
    "updateEvent": methods.updateEvent,
    "createRecurringEvent": methods.createRecurringEvent,
    "updateRecurringEvent": methods.updateRecurringEvent,
    "deleteRecurringEvent": methods.deleteRecurringEvent,
    "inviteEmail": methods.inviteEmail,
}

# Iterate through the function calls
for call in function_calls:
    # Properly parse the call to extract function name and arguments
    if "(" in call and ")" in call:
        function_name, arguments = call.split('(', 1)
        arguments = arguments.rsplit(')', 1)[0].split(', ')
        
        # Call the corresponding function with the provided arguments
        if function_name in functions:
            functions[function_name](*arguments)
        else:
            print(f"Function '{function_name}' not found.")



['createEvent("Meeting", "2024-04-18", "09:00", 1:00)']
Create event called
The data and time is  2024-04-18 / 09:00
UTC start time:  2024-04-18 13:00:00+00:00 UTC end time:  2024-04-18 14:00:00+00:00
Event created: Meeting on 2024-04-18 at 09:00 for duration of 1:00 hours.
